In [1]:
import pandas as pd
import xgboost as xgb
from numpy import array
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import seaborn as sn
import matplotlib.pyplot as plt
from xgboost import cv
from xgboost import XGBClassifier

In [2]:
data_frame = pd.read_csv("vista XGB 2021 v1.csv",delimiter=";")
data_frame.head(5)

,sexo,tos,fiebre,dificultad_respiratoria,fatiga,odinofagia,cefalea,rinorrea,diarrea,hipertension,diabetes,obesidad,enfermedades_respiratorias,resultado
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [3]:
df_train, df_test = train_test_split(data_frame, test_size=.2, random_state=1999)
data_frame["resultado"].value_counts()

1    418
0    192
Name: resultado, dtype: int64

In [4]:
df_train_mat = xgb.DMatrix(df_train.drop("resultado", 1), label=df_train["resultado"])
df_test_mat = xgb.DMatrix(df_test.drop("resultado", 1), label=df_test["resultado"])

In [6]:
params = {
    "learning_rate": 0.01,
    "max_depth": 3
}
# training, we set the early stopping rounds parameter
model_xgb = xgb.train(params, df_train_mat, evals=[(df_train_mat, "train"), (df_test_mat, "validation")], 
          num_boost_round=100, early_stopping_rounds=20)

[0]	train-rmse:0.49906	validation-rmse:0.49900
[1]	train-rmse:0.49814	validation-rmse:0.49802
[2]	train-rmse:0.49723	validation-rmse:0.49706
[3]	train-rmse:0.49634	validation-rmse:0.49611
[4]	train-rmse:0.49546	validation-rmse:0.49518
[5]	train-rmse:0.49461	validation-rmse:0.49431
[6]	train-rmse:0.49376	validation-rmse:0.49341
[7]	train-rmse:0.49293	validation-rmse:0.49257
[8]	train-rmse:0.49212	validation-rmse:0.49178
[9]	train-rmse:0.49132	validation-rmse:0.49105
[10]	train-rmse:0.49053	validation-rmse:0.49021
[11]	train-rmse:0.48975	validation-rmse:0.48951
[12]	train-rmse:0.48899	validation-rmse:0.48869
[13]	train-rmse:0.48825	validation-rmse:0.48801
[14]	train-rmse:0.48751	validation-rmse:0.48723
[15]	train-rmse:0.48679	validation-rmse:0.48657
[16]	train-rmse:0.48608	validation-rmse:0.48581
[17]	train-rmse:0.48539	validation-rmse:0.48518
[18]	train-rmse:0.48470	validation-rmse:0.48444
[19]	train-rmse:0.48403	validation-rmse:0.48384
[20]	train-rmse:0.48337	validation-rmse:0.48316
[2

In [7]:
model_xgb.best_ntree_limit

100

In [9]:
parametros = {
                "learning_rate": 0.01,
    "max_depth": 3
             }


"""estimator = XGBClassifier(
    objective= 'binary:logistic',
    booster="gbtree", 
    scale_pos_weight = 0.45,
    nthread=4,
    seed=42
)

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parametros,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)"""

#grid_search.fit(df_train_mat, "resultado")

rondas = 100
evaluacion = [(df_test_mat, "eval"), (df_train_mat, "train")]
modelo = xgb.train(parametros, df_test_mat, rondas, evaluacion) 

prediccion = modelo.predict(df_test_mat)
prediccion = [1 if i > .5 else 0 for i in prediccion]


[0]	eval-rmse:0.49802	train-rmse:0.49903
[1]	eval-rmse:0.49607	train-rmse:0.49808
[2]	eval-rmse:0.49415	train-rmse:0.49717
[3]	eval-rmse:0.49225	train-rmse:0.49628
[4]	eval-rmse:0.49039	train-rmse:0.49541
[5]	eval-rmse:0.48855	train-rmse:0.49457
[6]	eval-rmse:0.48674	train-rmse:0.49376
[7]	eval-rmse:0.48496	train-rmse:0.49297
[8]	eval-rmse:0.48321	train-rmse:0.49221
[9]	eval-rmse:0.48148	train-rmse:0.49147
[10]	eval-rmse:0.47978	train-rmse:0.49075
[11]	eval-rmse:0.47811	train-rmse:0.49006
[12]	eval-rmse:0.47646	train-rmse:0.48939
[13]	eval-rmse:0.47484	train-rmse:0.48873
[14]	eval-rmse:0.47324	train-rmse:0.48812
[15]	eval-rmse:0.47167	train-rmse:0.48752
[16]	eval-rmse:0.47012	train-rmse:0.48695
[17]	eval-rmse:0.46859	train-rmse:0.48638
[18]	eval-rmse:0.46709	train-rmse:0.48584
[19]	eval-rmse:0.46561	train-rmse:0.48531
[20]	eval-rmse:0.46415	train-rmse:0.48481
[21]	eval-rmse:0.46272	train-rmse:0.48432
[22]	eval-rmse:0.46131	train-rmse:0.48386
[23]	eval-rmse:0.45991	train-rmse:0.48340
[2

In [10]:
def metricas(objetivo, prediccion):

    matriz_conf = confusion_matrix(objetivo, prediccion)
    score = accuracy_score(objetivo, prediccion)
    reporte = classification_report(objetivo, prediccion)
    metricas = [matriz_conf, score, reporte]
    return(metricas)

In [11]:
met = metricas(df_test["resultado"], prediccion)

In [13]:
modelo.save_model("model_sklearn.json")

NameError: name 'model' is not defined

In [44]:
print(met[2])

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       125
           1       0.89      0.94      0.91       142

    accuracy                           0.90       267
   macro avg       0.90      0.90      0.90       267
weighted avg       0.90      0.90      0.90       267



In [12]:
params = {"objective":"binary:logistic",'learning_rate': 0.2,
                'max_depth': 10}

data_dmatrix = data_frame
data_frain_cv = xgb.DMatrix(data_dmatrix.drop("resultado", 1), label=data_dmatrix["resultado"])
          
xgb_cv = cv(dtrain=data_frain_cv, params=params, nfold=5,
                    num_boost_round=50, early_stopping_rounds=100, metrics="", as_pandas=True, seed=1999)
          
xgb_cv

XGBoostError: [18:24:52] C:\Users\Administrator\workspace\xgboost-win64_release_1.5.1\src\metric\metric.cc:49: Unknown metric function 

In [5]:
a= range(2, 10, 1)
a

range(2, 10)